In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
ra = 'all'
dec = 'all'
#ra = 270
#dec = -35
time = 'now'
bands = 'g'

# What's the predicted skybrightness?

In [ ]:
import os
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, AltAz, get_body
from astropy.time import Time
import astropy.units as u

from rubin_scheduler.scheduler.utils import generate_all_sky
from rubin_scheduler.utils import Site
from rubin_sim.skybrightness import SkyModel
import rubin_sim.phot_utils as phot_utils

import healpy as hp
import matplotlib.pyplot as plt

In [ ]:
if 'usdf' in os.getenv("EXTERNAL_INSTANCE_URL", ""):
        os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
if time == 'now':
    obs_time = Time.now()
else:
    obs_time = Time(time)

if ra == 'all' or dec == 'all':
    nside = 64
    sky = generate_all_sky(nside=nside, elevation_limit=20)
    ra_val = sky['ra']
    dec_val = sky['dec']
    all_sky = True
else:
    ra_val = ra
    dec_val = dec
    all_sky = False

if bands == 'all':
    bands = "ugrizy"
else:
    if isinstance(bands, str):
        if len(bands) == 1:
            bands = [bands]

In [ ]:
lsst_site = Site("LSST")
skybg_model = SkyModel(mags=True)

lst = obs_time.sidereal_time('apparent', lsst_site.to_earth_location())

aa = AltAz(location = lsst_site.to_earth_location(), obstime=obs_time)
skycoords = SkyCoord(ra=ra_val * u.deg, dec=dec_val * u.deg, frame="icrs")
# skycoords_gcrs = SkyCoord(ra=ra_val * u.deg, dec=dec_val * u.deg, 
#                           obstime=obs_time, location=lsst_site.to_earth_location(), frame="gcrs")
altaz = skycoords.transform_to(aa)

print(f"Checking @ obs_time = {obs_time.iso}, LST {lst :.3f}")
if all_sky: 
    print(f"For the whole sky.")
else:
    print(f"For RA {ra_val} and Dec {dec_val}")
    
    print(f"This point is at an airmass of {altaz.secz :.2f}")

In [ ]:
sun_coords = get_body('sun', obs_time, lsst_site.to_earth_location())
moon_coords = get_body('moon', obs_time, lsst_site.to_earth_location())
sun_altaz = sun_coords.transform_to(aa)
moon_altaz = moon_coords.transform_to(aa)

print(f"Sun at altitude {sun_altaz.alt.deg :.1f}")
print(f"Moon at alt/az {moon_altaz.alt.deg :.1f} {moon_altaz.az.deg : .1f}")

if not all_sky:
    distance_to_moon = altaz.separation(moon_altaz)
    print(f"The moon is {distance_to_moon.deg : .1f} degrees away.")

In [ ]:
# Calculate sky background.
skybg_model.set_ra_dec_mjd(ra_val, dec_val, obs_time.mjd, degrees=True)
sky_mags = skybg_model.return_mags()

In [ ]:
platescale = 0.2
exptimes = np.arange(14, 40, 2)
idx = np.where(exptimes == 30)[0]
sky_counts = {}
for b in 'ugrizy':
    sky_counts[b] = np.zeros((len(exptimes), len(sky_mags[b])))
    for i, exptime in enumerate(exptimes):
        zp_sky = phot_utils.predicted_zeropoint_hardware(b, exptime) 
        sky_counts[b][i, :] = np.power(10, (sky_mags[b] - zp_sky)/-2.5) * platescale**2

In [ ]:
if all_sky:
    rot = (lst.deg, lsst_site.latitude, 0)
    for b in bands:  
        hp.mollview(sky_mags[b], rot=rot, title=f"Sky Mags {b}")
        hp.graticule(dpar=30, dmer=60, alpha=0.3)
        hp.projscatter(moon_coords.ra, moon_coords.dec, color='b', label='Moon', lonlat=True)
        hp.projscatter(sun_coords.ra, sun_coords.dec, color='y', label='Sun', lonlat=True)
        _  = plt.legend(loc=(1.01, 0.5))
    print("Plot of background sky brightness in mag/arcsecond^2, RA/Dec grid, centered on location of Rubin")

In [ ]:
band_colors_white = {
        "u": "#0c71ff",
        "g": "#49be61",
        "r": "#c61c00",
        "i": "#ffc200",
        "z": "#f341a2",
        "y": "#5d0000",
    }
plt.figure(figsize=(8,6))
for b in bands:
    if all_sky:
        plt.plot(exptimes, np.nanmax(sky_counts[b], axis=1), color=band_colors_white[b], linestyle=':', label=f"{b} max counts/pixel")
        plt.plot(exptimes, np.nanmean(sky_counts[b], axis=1), color=band_colors_white[b], linestyle='-', label=f"{b} mean counts/pixel")
        plt.plot(exptimes, np.nanmin(sky_counts[b], axis=1), color=band_colors_white[b], linestyle='-.', label=f"{b} min counts/pixel")
    else:
        plt.plot(exptimes, sky_counts[b], color=band_colors_white[b], linestyle='-', label=f"{b} sky counts")
plt.xlabel("Exposure time (s)")
plt.ylabel("sky counts (electrons/pixel)")
plt.legend(loc=(1.01, 0.1))
plt.grid(True, alpha=0.3, color='gray')

In [ ]:
if all_sky:
    print("Describe sky_mags")
    sky_df = pd.DataFrame(sky_mags)
    display(sky_df.describe())
    print("Describe sky counts")
    sky_df = pd.DataFrame(dict([(b, sky_counts[b][idx, :][0]) for b in 'ugrizy']))
    display(sky_df.describe())
else:
    sky_df = pd.concat([pd.DataFrame(sky_mags, index=["Mags"]), pd.DataFrame(dict([(b,sky_counts[b][idx, 0]) for b in 'ugrizy']), index=["Counts"])])
    display(sky_df.round(2))